In [17]:
import json
import random
import sys
import os

# Add the path to the directory containing the attribute_cleaner module
sys.path.append(os.path.abspath("../../transformer"))

# Now you can import text_cleaner
from attribute_cleaner.general_string_cleaner import text_cleaner

#### check MPS(metal performance shraders) - apple

In [2]:
import torch

# Check that MPS is available
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    print("all good")

all good


#### check CUDA

In [3]:
import torch

if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")


CUDA is not available.


In [4]:
train_path = "../data/merged_data.json"

with open(train_path, "r") as f:
    data = json.load(f)

print('Dataset size:', len(data))

random.shuffle(data)
print('Dataset is shuffled...')

train_dataset = data[:int(len(data)*0.9)]
test_dataset = data[int(len(data)*0.9):]

print('Dataset is splitted...')

Dataset size: 183
Dataset is shuffled...
Dataset is splitted...


In [5]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

import torch
from gliner import GLiNERConfig, GLiNER
from gliner.training import Trainer, TrainingArguments
from gliner.data_processing.collator import DataCollatorWithPadding, DataCollator
from gliner.utils import load_config_as_namespace
from gliner.data_processing import WordsSplitter, GLiNERDataset

In [6]:
device = torch.device('mps')
model = GLiNER.from_pretrained("urchade/gliner_small")

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/Users/snehil/.dev_env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Users/snehil/.dev_env/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/Users/snehil/.dev_env/lib/python3.12/site-packages/gliner/model.py:794: FutureWarning: 

In [7]:
# use it for better performance, it mimics original implementation but it's less memory efficient
data_collator = DataCollator(model.config, data_processor=model.data_processor, prepare_labels=True)

In [8]:
# Optional: compile model for faster training
model.to(device)
print("done")

done


In [9]:
# calculate number of epochs
num_steps = 30
batch_size = 8
data_size = len(train_dataset)
num_batches = data_size // batch_size
num_epochs = max(1, num_steps // num_batches)

training_args = TrainingArguments(
    output_dir="models",
    learning_rate=5e-6,
    weight_decay=0.01,
    others_lr=1e-5,
    others_weight_decay=0.01,
    lr_scheduler_type="linear", #cosine
    warmup_ratio=0.1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    focal_loss_alpha=0.75,
    focal_loss_gamma=2,
    num_train_epochs=num_epochs,
    evaluation_strategy="steps",
    save_steps = 100,
    save_total_limit=10,
    dataloader_num_workers = 0,
    use_cpu = False,
    report_to="none",
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=model.data_processor.transformer_tokenizer,
    data_collator=data_collator,
)

trainer.train()

/Users/snehil/.dev_env/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/21 [00:00<?, ?it/s]

/Users/snehil/.dev_env/lib/python3.12/site-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 425 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/Users/snehil/.dev_env/lib/python3.12/site-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 563 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/Users/snehil/.dev_env/lib/python3.12/site-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 495 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/Users/snehil/.dev_env/lib/python3.12/site-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 416 has been t

{'train_runtime': 81.2102, 'train_samples_per_second': 2.019, 'train_steps_per_second': 0.259, 'train_loss': 97.60902622767857, 'epoch': 1.0}


TrainOutput(global_step=21, training_loss=97.60902622767857, metrics={'train_runtime': 81.2102, 'train_samples_per_second': 2.019, 'train_steps_per_second': 0.259, 'total_flos': 0.0, 'train_loss': 97.60902622767857, 'epoch': 1.0})

##### `Note: Research on how num_steps and batch size works , it affects the quality of model a lot`

In [10]:
trained_model = GLiNER.from_pretrained("models/checkpoint-21", load_tokenizer=True)

config.json not found in /Users/snehil/Work/self/coffee/coffee_database/ner/GLiNER/models/checkpoint-21
/Users/snehil/.dev_env/lib/python3.12/site-packages/gliner/model.py:794: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for a

In [20]:
text = """
Single origin Indian Coffee From Moganad Estate, Tamil Nadu which is a 100% Arabiaca coffee with 
an exquisite blend of balanced sweetness and brightness. A Medium Dark Roast coffee with 
flavor notes of Cocoa, Caramel and Nut which can also be enjoyed in a French press, moka pot, aeropress & espresso .
It is washed processed, grown at an altitude of around 4430 ft
"""

# Labels for entity prediction
labels = ['BREWING_DEVICE','COFFEE_PROPERTIES','ELEVATION','ESTATE','PROCESSING','ROAST LEVEL','VARIETAL','FERMENTATION TIME',
'TASTING NOTES','LOCATION','AROMA','BODY','ACIDITY','ROASTER','FARMER','TEXTURE','COFFEE TYPE'
] # for v2.1 use capital case for better performance

# Perform entity prediction
entities = trained_model.predict_entities(text_cleaner(text), labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

moganad estate => ESTATE
tamil nadu => LOCATION
sweetness => TASTING NOTES
brightness => TASTING NOTES
medium => ROAST LEVEL
cocoa => TASTING NOTES
caramel => TASTING NOTES
nut => TASTING NOTES
french press => PROCESSING
moka pot => BREWING_DEVICE
aeropress => PROCESSING
espresso => PROCESSING
washed => PROCESSING
altitude => ELEVATION
4430 ft => ELEVATION


In [27]:
text = """
Jasmine kissed cranberry - 100 % arabica that is sourced from Chikmagalur, grown at an altitude of approximately 4,100ft to 4,500ft.
It gets its unique name through a process which include a strain of yeast used to create a 
carbon dioxide-rich environment during fermentation in stainless-steel fermenters. 
A remarkable level of complexity in the beans is created by this process, laying the groundwork for 
a wonderful cup of coffee. Steady drying on raised beds is the next important step, 
which lets the flavors gradually develop and intensify. After another 30 days of continuous stirring, 
the coffee undergo additional drying developing scent of jasmine, complex notes of raspberries, 
cranberries, and sparkling malic acidity with a lingering floral aftertaste.
"""

# Labels for entity prediction
labels = ['BREWING_DEVICE','COFFEE_PROPERTIES','ELEVATION','ESTATE','PROCESSING','ROAST LEVEL','VARIETAL','FERMENTATION TIME',
'TASTING NOTES','LOCATION','AROMA','BODY','ACIDITY','ROASTER','FARMER','TEXTURE','COFFEE TYPE'
] # for v2.1 use capital case for better performance

# Perform entity prediction
entities = trained_model.predict_entities(text_cleaner(text), labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

chikmagalur => LOCATION
altitude => ELEVATION
yeast => COFFEE TYPE
carbon dioxide => ACIDITY
fermentation => PROCESSING
drying => PROCESSING
raised beds => LOCATION
30 days => FERMENTATION TIME
continuous stirring => PROCESSING
drying => PROCESSING
jasmine => TASTING NOTES
raspberries => TASTING NOTES
cranberries => TASTING NOTES
malic acidity => ACIDITY


In [28]:
text = """
Anuswaram in Carnatic parlance means a note between notes and denotes complex melodic movements that rāga-s tend to employ.
We sometimes receive small lots of coffee that really captivate us with their complexity or innovative processing. 
We feel that Anuswaram is a fitting name for featuring these intriguing offerings. 
The 4th in this series is a very special washed coffee from the lofty hills of Valparai.
Located in the Thalanar region of the Valparai plateau, Greenland estate is spread over 200 acres and at an
 altitude of 1500- 1737m ASL, is one of the highest farms that we have sourced from. 
 The owner Mr. Govind, is very eager to make inroads into the speciality coffee segment and nothing showcases what 
 the farm has to offer like this washed lot.
To prepare this, cherries of the SLN 09 varietal were picked picked ripe, pulped (i.e. skin was removed) and 
fermented for 18 hours. The coffee beans were  then washed  and slow dried for a fortnight on raised beds.
The low temperatures(because  of the high altitude) and the care taken during processing have made this coffee an 
absolute delight in the cup! There's a very prominent jaggery like sweetness, a partly peachy , 
partly orange-like acidity and a whole bunch of florals, some of.which we could triangulate and 
some which we couldn't. All this complexity is very reminiscent of African coffees and quite uncommon in Indian offerings. 
We thank Mr. Viggnesh for having helped source this beautiful coffee for us and hope to work with 
Greenland estate more closely in future!
Anuswaram #4 is a delight across all manual brews but shines best in a v60.
"""

# Labels for entity prediction
labels = ['BREWING_DEVICE','COFFEE_PROPERTIES','ELEVATION','ESTATE','PROCESSING','ROAST LEVEL','VARIETAL','FERMENTATION TIME',
'TASTING NOTES','LOCATION','AROMA','BODY','ACIDITY','ROASTER','FARMER','TEXTURE','COFFEE TYPE'
] # for v2.1 use capital case for better performance

# Perform entity prediction
entities = trained_model.predict_entities(text_cleaner(text), labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

carnatic => ELEVATION
processing => PROCESSING
valparai => LOCATION
thalanar => LOCATION
valparai => ESTATE
greenland estate => ESTATE
200 acres => ELEVATION
altitude => ROAST LEVEL
mr . govind => FARMER
fermented => PROCESSING
18 hours => FERMENTATION TIME
washed => PROCESSING
slow dried => PROCESSING
fortnight => FERMENTATION TIME
high altitude => ROAST LEVEL
processing => PROCESSING
jaggery => ACIDITY
sweetness => TASTING NOTES
peachy => ACIDITY
orange => ACIDITY
acidity => TASTING NOTES
florals => TASTING NOTES
african => ELEVATION
indian => ELEVATION
mr . viggnesh => FARMER
greenland estate => ESTATE


#### not that bad, but lot of room for improvement